In [1]:
from graphframes import *
from pyspark import *
from pyspark.sql import *

In [2]:
import findspark

#findspark.init()

sc = SparkContext()
sqlContext = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at C:\Users\Hiresh\Anaconda3\lib\site-packages\IPython\utils\py3compat.py:188 

In [3]:
df_wordgame = sqlContext.read.format("csv").option("header", "true")\
.csv('C:/Users/Hiresh/Desktop/UMKC/Big Data Prog/lab2_task4/wordgame_20170721.csv')

In [4]:
df_wordgame.registerTempTable("wordgame")

In [5]:
df_wordgame.show()

+------+------------+------------+------+--------+
|author|       word1|       word2|source|sourceID|
+------+------------+------------+------+--------+
|  1353|       email|        spam|   GOG|       4|
|  1568|  highlander|  zweihander|   GOG|       4|
|  4793|    dictator|    decision|    U2|       8|
|   796|       horse|      cowboy|   ECF|       3|
|   289|      maudie|     atticus|    AC|       0|
|  2353|   substance|      liquid|   LEF|       5|
|  2696|     numbers|    quantity|   SAS|       6|
|  1113|     endgame|      battle|   GOG|       4|
|  1079| machinarium|   mechanism|   GOG|       4|
|  4726|        star|       light|    U2|       8|
|   312|      limpet|        mine|    AC|       0|
|  1041|        pick|         hoe|   ECF|       3|
|  4587|        name|      maiden|    TF|       7|
|   390|substantiate|authenticate|    BC|       1|
|  5342|      ibutts|        ipoo|    WP|       9|
|   883|     privacy|    solitude|   ECF|       3|
|  5726|        walk|        ma

In [6]:
after_duplicate=sqlContext.sql('select author,word1 as id,word2,source,sourceID from wordgame \
group by  author,word1,word2,source,sourceID limit 1000')
after_duplicate.registerTempTable("newwordgame")

In [9]:
vertices=sqlContext.sql('select author,id,word2,source,sourceID from newwordgame \
group by  author,id,word2,source,sourceID')

In [10]:
edges=sqlContext.sql('select id as src,word2 as dst, source as source from newwordgame group by id,word2,source')

In [11]:
from graphframes import *
graph=GraphFrame(vertices,edges)

In [12]:
pageRank=graph.pageRank(resetProbability=0.15,maxIter=1)

In [13]:
pageRank.vertices.show()

+------+-----------------+------------+------+--------+------------------+
|author|               id|       word2|source|sourceID|          pagerank|
+------+-----------------+------------+------+--------+------------------+
|   312|        broadcast|transmission|    AC|       0|0.5327695560253642|
|  4522|           boring|    politics|    TF|       7|0.5327695560253642|
|  1317|            party|       house|   GOG|       4| 3.551797040169095|
|  6272|            party|        gala|    WP|       9| 3.551797040169095|
|  4240|            avoid|      devoid|   SAS|       6|0.5327695560253642|
|  4522|         bungalow|        flat|    TF|       7|0.5327695560253642|
|  4762|mocha choco latte|  choco taco|    U2|       8|0.5327695560253642|
|   873|             sops|       gravy|   ECF|       3|0.5327695560253642|
|  2525|           blammo|         pow|   SAS|       6|0.5327695560253642|
|   387|          pickled|   cucumbers|    BC|       1|0.5327695560253642|
|    39|        criminals

In [14]:
pageRank.edges.show()

+----------+-------------+------+------+
|       src|          dst|source|weight|
+----------+-------------+------+------+
|flatulence|          gas|    TF|   1.0|
|        mr|           mr|    AC|   1.0|
|      tree|       leaves|    WP|   0.5|
|      tree|       leaves|    WP|   0.5|
|      tree|       leaves|    WP|   0.5|
|      tree|       leaves|    WP|   0.5|
|membership|         card|   GOG|   0.5|
|membership|         card|   GOG|   0.5|
|membership|         card|   GOG|   0.5|
|membership|         card|   GOG|   0.5|
|      edge|      science|    WP|   1.0|
|       tnt|     dynamite|   ECF|   1.0|
|      poke|         fire|    TF|   1.0|
|     shade|     coolness|    WP|   1.0|
|  airfield|         down|    WP|   1.0|
|    change|metamorphosis|    CC|   1.0|
|    apples|          red|    WP|   1.0|
|   barrier|        cream|   SAS|   1.0|
| shootings|     horrible|    TF|   1.0|
|       hat|        cover|   SAS|0.0625|
+----------+-------------+------+------+
only showing top